In [ ]:
!pip install praw pandas duckdb

   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   ---------------------------------------- 11.4/11.4 MB 54.8 MB/s eta 0:00:00


In [2]:
!pip install datetime

  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
Using cached DateTime-5.5-py3-none-any.whl (52 kB)

   ---------------------------------------- 0/2 [zope.interface]
   ---------------------------------------- 0/2 [zope.interface]
   ---------------------------------------- 0/2 [zope.interface]
   ---------------------------------------- 0/2 [zope.interface]
   -------------------- ------------------- 1/2 [datetime]
   ---------------------------------------- 2/2 [datetime]



In [ ]:
import os
print(os.getcwd())

f:\U\visualizacion\final


In [ ]:
import pandas as pd
import praw
import datetime
import numpy as numpy
import duckdb

In [ ]:
#credencial 
reddit = praw.Reddit(
                                    client_id="BlITQKHTmAdDpt-RuC3Xlw", 
                                    client_secret="gVHTWcDqVl_MJsYcMl_UadJcLFw4lA", 
                                    user_agent="banshhe02"
                                    )

In [ ]:
# subreddits
mangacollectors="MangaCollectors"
manga="manga"
anime="anime"

#funcion obtiene informacion de un subreddit y la muestra
def info_subreddit(subreddit):
    print(f"Nombre completo: r/{subreddit.display_name}")
    print(f"Título: {subreddit.title}")
    print(f"Descripción: {subreddit.public_description}")
    print(f"Suscriptores: {subreddit.subscribers:,}")
    print(f"Usuarios activos: {subreddit.active_user_count:,}")
    print(f"Fecha de creación: {datetime.datetime.fromtimestamp(subreddit.created_utc).strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
#toma la informacion de los subreddit y la muestra usando la funcion definida arriba
info_subreddit(reddit.subreddit("MangaCollectors"))
info_subreddit(reddit.subreddit("manga"))
info_subreddit(reddit.subreddit("anime"))

Nombre completo: r/MangaCollectors
Título: Manga Collectors
Descripción: #####Show off your manga collections and pickups. Discussion posts are encouraged so feel free to create topics about publishers, quality of translations, quality of physical books, excitement about a recent pickup, and more. You do not need to be a collector to participate in this community.

---

**IMPORTANT LINKS**

[Where to Buy Manga](https://www.reddit.com/r/MangaCollectors/comments/cezrxy/best_places_to_buy_manga_for_each_region/)
Suscriptores: 1,905,371
Usuarios activos: 38
Fecha de creación: 2017-10-15 19:03:23
Nombre completo: r/manga
Título: /r/manga: manga, on reddit.
Descripción: Everything and anything manga!  (manhwa/manhua is okay too!)

Discuss weekly chapters, find/recommend a new series to read, post a picture of your collection, lurk, etc!
Suscriptores: 4,722,122
Usuarios activos: 1,614
Fecha de creación: 2008-08-09 16:35:11
Nombre completo: r/anime
Título: /r/Anime
Descripción: Reddit's premie

In [ ]:
#estructura para almacenar los datos
datos = {
    'id_post': [],
    'titulo_post': [],
    "fecha_post": [],
    "autor_post": [],
    "URL": [],
    "numero_comentarios": [],
    "texto_post": [],
    "tiene_texto": [],
    "puntaje": [],     
    "upvotes": [],    
    "downvotes": []   
}

In [9]:
def extraccion_info_post(post):
    fecha = datetime.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    autor = str(post.author) if post.author else "ELIMINADO"
    tiene_texto = "Sí" if post.selftext else "No"

    datos['id_post'].append(post.id)
    datos['titulo_post'].append(post.title)
    datos['fecha_post'].append(fecha)
    datos['autor_post'].append(autor)
    datos['URL'].append(post.url)
    datos['numero_comentarios'].append(post.num_comments)
    datos['texto_post'].append(post.selftext)
    datos['tiene_texto'].append(tiene_texto)
    datos['puntaje'].append(post.score)
    datos['upvotes'].append(post.ups)
    datos['downvotes'].append(post.downs)

In [10]:
subreddits_a_procesar = ["MangaCollectors", "manga", "anime"]
limit_por_subreddit = 1000
time_filter_period = 'year'

for subreddit_nombre in subreddits_a_procesar:
    sub = reddit.subreddit(subreddit_nombre)
    for submission in sub.top(limit=limit_por_subreddit, time_filter=time_filter_period):
        extraccion_info_post(submission)

dataframe

In [ ]:
df_posts = pd.DataFrame(datos)

In [ ]:
print(df_posts.head())

   id_post                                        titulo_post  \
0  1esb25s          First time Posting My 7 year collection!    
1  1fjfdpg  I’m convinced Barnes & Noble have no idea what...   
2  1d752xq                     Maybe I Should Do a Room Tour?   
3  1fp5nlw  Just moved into my new house and had to recrea...   
4  1d4fojl                            I see this a lot here.    

            fecha_post         autor_post  \
0  2024-08-14 15:57:13       YiXiao_Yksha   
1  2024-09-17 21:13:04       sendlewdzpls   
2  2024-06-03 09:58:06         Wismur10Hp   
3  2024-09-25 11:22:05  The_Manga_Monster   
4  2024-05-30 18:04:18       jayvancealot   

                                      URL  numero_comentarios  \
0  https://www.reddit.com/gallery/1esb25s                 239   
1    https://i.redd.it/kykkfhjxmgpd1.jpeg                 191   
2  https://www.reddit.com/gallery/1d752xq                 253   
3  https://www.reddit.com/gallery/1fp5nlw                 175   
4     https:/

In [ ]:
df_posts.to_csv('posts.csv', index=False, encoding='utf-8')